# `Meltwater-HateSpeeech Data Analysis`
The analysis covers twitter data and expose the malice behind ampilification of content that results to trennding content and hashtags. The data is from [Twitter](https://twitter.com/) scrapped through [Meltwater](https://app.meltwater.com/home) platform and API.
 
 The analysis seeks to answer whether the content shared is real or not or is there a motive behind the sharing and amplifying of content on Twitter.

 ## `Environment Setup`
 Using the jupyter notebook in conda environment, install requirements via terminal
 ```py
 conda install --file requirements.txt
 ```

 ### Install Requirements
 Ensure that the requirements.txt file is present in the working folder.
 Note: that I am using a python 3.10.9 kernel.

In [13]:
%pip install -r requirements.txt #Install packages

  Using cached numpy-1.24.2-cp310-cp310-macosx_11_0_arm64.whl (13.9 MB)

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import relevant Libraries or access Keys

In [14]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import seaborn as sns
import csv
# import datatable as dt
import seaborn as sns
pd.set_option('display.max_colwidth', None)
from IPython.display import Image
from collections import Counter
import re
import codecs

## `Data`
### Import the data

In [16]:
df = pd.read_csv("hatespeech tweets 2022.csv")

### Basic data properties

In [22]:
df.shape

(20000, 38)

The data has 20000 tweets with 38 variables.

A brief description of the data and the data types found in the data can be extracted by the function _`info()`_

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   20000 non-null  object 
 1   Headline               0 non-null      float64
 2   URL                    20000 non-null  object 
 3   Opening Text           0 non-null      float64
 4   Hit Sentence           20000 non-null  object 
 5   Source                 20000 non-null  object 
 6   Influencer             20000 non-null  object 
 7   Country                20000 non-null  object 
 8   Subregion              0 non-null      float64
 9   Language               20000 non-null  object 
 10  Reach                  20000 non-null  int64  
 11  Desktop Reach          20000 non-null  int64  
 12  Mobile Reach           20000 non-null  int64  
 13  Twitter Social Echo    0 non-null      float64
 14  Facebook Social Echo   0 non-null      float64
 15  Re

Since we would like to determine 'why the content shared' lets look at the consentiments and the content shared and there sentiments:

In [17]:
df[['Hit Sentence', 'Sentiment']]

,Hit Sentence,Sentiment
0,RT @julius_muiruri: Hatupangwingwi. ; Consumers will be hit by an electricity price shock in January 2023 as the subsidy which retired president Uhuru Kenyatta initiated is dropped. https://t.co/dt6jEfQruY,Negative
1,RT @julius_muiruri: Hatupangwingwi. ; Consumers will be hit by an electricity price shock in January 2023 as the subsidy which retired president Uhuru Kenyatta initiated is dropped. https://t.co/dt6jEfQruY,Negative
2,"QT @OmKaYoo: so? ; Twitter feed today is filled with skid and ruto being horny, mouth watering over breaking bad, or getting pissed at and threatening to hurt/kill absolutely nobody",Negative
3,"@NjiruAdv @rigathi @WilliamsRuto Bitterness will kill this ""Black Bull"" @NjiruAdv. What did Riggy G and Samoei son of Ruto ever do to you other than beating your candidates, great grandpa @RailaOdinga and grandma @MarthaKarua hands down? Continued barking doesn't add you cleints; Actual working does!",Negative
4,RT @julius_muiruri: Hatupangwingwi. ; Consumers will be hit by an electricity price shock in January 2023 as the subsidy which retired president Uhuru Kenyatta initiated is dropped. https://t.co/dt6jEfQruY,Negative
...,...,...
19995,@WilliamsRuto We believe as team UDA that Kenya is greater and speaks louder than any other self-proclaimed opposition president whose political interests are based on thwarfting upcoming young politicians@ Raila's followers are blind to see that they are being used kill abilities.........,Negative
19996,RT @joekaranja91: PRESIDENT Uhuru Kenyatta has asked the GEMA community to remember just one thing he has done for them. Personally I'll never forget how the KRA and KPA delays led to the collapse of my business. I'll not forget how per cbm prices were doubled to kill our investment.,Negative
19997,RT @joekaranja91: PRESIDENT Uhuru Kenyatta has asked the GEMA community to remember just one thing he has done for them. Personally I'll never forget how the KRA and KPA delays led to the collapse of my business. I'll not forget how per cbm prices were doubled to kill our investment.,Negative
19998,@AldrinSimbaa @LochRapemo @MosesMw10357761 @MutahiNgunyi Depression will kill you guys cz you can't keep on telling Kenyans...is our freedom fighter yet he has no record of such.I know raila as a very selfish person I mean he can't share anything maybe if that one thing is for his personal gain anything else,Negative


You will observe that from an overview we have most retweets with negative sentiments.

One major focus of the analysis is natural language processing where we focus of the tweets content in determining similarities in sharing of original content.

For starters let's rename the 'Hit Sentence' to tweet and Influencer to target:

In [23]:
df.rename(columns={'Hit Sentence' :'tweet'},inplace=True)
df.rename(columns={'Influencer' :'target'},inplace=True)

## `Language Analysis`
### Original Tweets
While there are 20000 tweets how many original tweets do we have? These are the tweeets that do not have the phrase "RT"  to signify "retweeted" or "QT" for quoted tweets.

In [30]:
# Drop retweets
original_tweets = df[df["tweet"].str.startswith("RT") == False]

# Drop quoted tweets
original_tweets = original_tweets[original_tweets["tweet"].str.startswith("QT") == False]

# Original tweets
original_shape = original_tweets.shape
original_shape

(3512, 38)

**Result:** There are 3512 original tweets from 20000 tweets.

This could only mean that the 3512 tweets could have been retweeted multiple times.

The question is why? And is there a network of accounts meant to only retweet contents of a particular account?
Also are the accounts retweeting this content real people or are they bots?

### Unique Accounts
Of the 3512 original tweets, how many unique users to we have?

In [26]:
# Number of unique accounts
original_tweets['target'].nunique()

2626

The data contains 2626 unique users who shared content with hatespeech words.

### Keyword Search
One of the hatespeech words used and flagged by the NCIC during the elction period is the phrase "**_kill_**".

How many original tweets used this phrase in their content?

In [35]:
df_kill_original = original_tweets[original_tweets["tweet"].str.contains("kill")]
keyword_shape = df_kill_original.shape
print(keyword_shape)


percentage = (keyword_shape[0]/original_shape[0]) * 100
print("The percentage is:", percentage, "%")


(2337, 38)
The percentage is: 66.54328018223235 %


Note that 2337 of 3512 tweets used the phrase "**_kill_**". This is 66% of all the tweet

Why are 2337 people using a single word in their tweets? What is the impact of the same?

### Quoted Tweets
These are tweets that begin with the phrase "QT". How many are there that have the phrase "**_kill_**"?

In [41]:
quoted_tweets = df[df["tweet"].str.startswith("QT") == True]
df_kill_quoted = quoted_tweets[quoted_tweets["tweet"].str.contains("kill")]
df_kill_quoted.shape

(1077, 38)

There are 1077 quoted tweets with the phrase kill.

### Retweets
On Twitter, this is one of the major ways that accounts use to amplify content.

In [47]:
retweeted_tweets = df[df["tweet"].str.startswith("RT") == True]
print(retweeted_tweets.shape)

# Retweeted content
percentage = (retweeted_tweets.shape[0]/df.shape[0]) * 100
print("The Retweeted percentage is:", percentage, "%")

# Quoted content
percentage = (quoted_tweets.shape[0]/df.shape[0]) * 100
print("The Quoted percentage is:", percentage, "%")

original_tweets.shape
# Quoted content
percentage = (original_tweets.shape[0]/df.shape[0]) * 100
print("The Original percentage is:", percentage, "%")

(14952, 38)
The Retweeted percentage is: 74.76 %
The Quoted percentage is: 7.68 %
The Original percentage is: 17.560000000000002 %


There area 14952 retweets. 

From this sample we can conclude that 75% of content in Twiiter has been amplified through retweeting and only 17% of the Tweets are original.

In [49]:
%pip install sentence-transformers

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done


In [48]:
from sentence_transformers import SentenceTransformer

ModuleNotFoundError: No module named 'sentence_transformers'